In [119]:
import numpy as np
import cv2 
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format('channels_last')
import numpy as np
from keras.datasets import mnist #Keras is used only to get MNIST data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = Y_train.reshape(60000, 1).T 
b = np.zeros((60000, 10))
b[np.arange(60000), Y_train] = 1
Y_train = b #To convert to one hot matrix
print(Y_train.shape)
X_train = X_train.reshape(60000, 28, 28, 1)/255 #Divided by 255 to normalize the input data
print(X_train.shape)
Y_test = Y_test.reshape(10000, 1).T
b = np.zeros((10000, 10))
b[np.arange(10000), Y_test] = 1
Y_test = b
print(Y_test.shape)
X_test = X_test.reshape(10000, 28, 28, 1)/255
print(X_test.shape)

(60000, 10)
(60000, 28, 28, 1)
(10000, 10)
(10000, 28, 28, 1)


In [120]:
def moodel(input_shape):

    X_input = Input(input_shape)

  
    X = ZeroPadding2D((1, 1))(X_input)

    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    X = ZeroPadding2D((2, 2))(X_input)
    
    X = Conv2D(32, (5, 5), strides = (2, 2), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)

    
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    X = Flatten()(X)
    X = Dense(10, activation='softmax', name='fc')(X)
    model = Model(inputs = X_input, outputs = X, name='HappyModel')

    
    return model

In [121]:
moo = moodel(X_train.shape[1:4])
moo.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])
moo.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 128)

Epoch 1/10
60000/60000 [==============================] - 28s 473us/step - loss: 0.0441 - acc: 0.98551s 
Epoch 2/10
60000/60000 [==============================] - 27s 456us/step - loss: 0.0159 - acc: 0.9948
Epoch 3/10
60000/60000 [==============================] - 28s 471us/step - loss: 0.0121 - acc: 0.99604s - loss: 0.0120 -  - ETA: 3s - loss: 0.
Epoch 4/10
59392/60000 [============================>.] - ETA: 0s - loss: 0.0100 - acc: 0.9967

KeyboardInterrupt: 

In [122]:
import numpy as np
import cv2 as cv2

In [123]:
red = np.uint8([[[0,0,255 ]]])
hsv_red = cv2.cvtColor(red,cv2.COLOR_BGR2HSV)
print(hsv_red)

[[[  0 255 255]]]


In [133]:
cap = cv2.VideoCapture(0)
kernel = np.ones((5,5),np.uint8)
black = np.zeros((512,512,1), dtype=np.uint8)
points = []
while(1):
    k = cv2.waitKey(5) & 0xFF
    # Take each frame
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red = np.array([110,50,50])
    upper_red = np.array([130,255,255])
    img = cv2.inRange(hsv, lower_red, upper_red)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.erode(img, kernel, iterations=3)
    img = cv2.dilate(img, kernel, iterations = 2)
    contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        #Sorting of contours on the basis of area
        contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(contour)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        M = cv2.moments(contour)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        points.append(center)
        for i in range(1, len(points)):
            if points[i - 1] is None or points[i] is None:
                continue
            # Frame is for displaying
            cv2.line(black, points[i - 1], points[i], (255, 255, 255), 20)
            cv2.line(frame, points[i - 1], points[i], (0, 0, 0), 2)
    else :
        
        goo = cv2.resize(black, (28, 28), interpolation = cv2.INTER_CUBIC)
        #black = np.zeros((512,512,1), dtype=np.uint8)
        goo = np.array(goo)
        goo = goo.reshape(1, 28, 28, 1)/255
        Y_ = moo.predict(goo)
        Y = Y_.argmax(axis=1)
        #print(Y)
        cv2.putText(frame, "Prediction:  " + str(Y), (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        points = []
    #cv2.imshow('goo', img)
    if k==ord("a"):
        black = np.zeros((512,512,1), dtype=np.uint8)
    #cv2.putText(frame, "Convolution Neural Network:  " + str(Y), (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow('moo', black)
    cv2.imshow('original image', frame)
    if k == 27:
        break

cv2.destroyAllWindows()